In [269]:
import pandas as pd
import glob
import numpy as np
from prophet import Prophet
import plotly.graph_objects as go

In [270]:
dossier = "csv"
chemins_fichiers = glob.glob(f"{dossier}/b20[1-2][0-9].csv")
dfs = []
for chemin_fichier in chemins_fichiers:
    df = pd.read_csv(chemin_fichier)
    dfs.append(df)
df_final = pd.concat(dfs, ignore_index=True)
print(df_final)

            Date   Dernier      Ouv.  Plus Haut  Plus Bas     Vol. Variation %
0     31/12/2019   7.196,4   7.261,5    7.331,0   7.167,4  586,60K      -0,90%
1     30/12/2019   7.261,8   7.397,5    7.420,9   7.244,1  606,11K      -1,84%
2     29/12/2019   7.397,5   7.321,6    7.518,9   7.303,0  611,69K       1,04%
3     28/12/2019   7.321,5   7.261,9    7.375,9   7.256,5  610,96K       0,82%
4     27/12/2019   7.261,7   7.210,8    7.293,8   7.128,5  718,07K       0,70%
...          ...       ...       ...        ...       ...      ...         ...
1906  05/01/2021  33.991,5  32.015,4   34.414,7  30.010,5  202,13K       6,15%
1907  04/01/2021  32.022,6  33.016,6   33.587,5  28.204,5  255,27K      -2,84%
1908  03/01/2021  32.958,9  32.192,9   34.755,9  32.029,6  155,21K       2,38%
1909  02/01/2021  32.193,3  29.359,7   33.233,5  29.008,0  240,87K       9,65%
1910  01/01/2021  29.359,9  28.951,7   29.627,1  28.712,4  100,90K       1,42%

[1911 rows x 7 columns]


In [271]:
df_final.to_csv("csv/data.csv", index=False)

In [272]:
data = pd.read_csv('csv/data.csv') 
data.dropna(inplace=True)

data['Dernier'] = data['Dernier'].astype(str).str.replace('.', '').str.replace(',', '.').astype(float)
data['Ouv.'] = data['Ouv.'].astype(str).str.replace(',', '.').str.replace('.', '').astype(float)
data[' Plus Haut'] = data[' Plus Haut'].astype(str).str.replace(',', '.').str.replace('.', '').astype(float)
data['Plus Bas'] = data['Plus Bas'].astype(str).str.replace(',', '.').str.replace('.', '').astype(float)
data['Vol.'] = data['Vol.'].astype(str).str.replace(r'[^\d.]$', '', regex=True).str.replace(',', '.').astype(float)
data['Variation %'] = data['Variation %'].astype(str).str.rstrip('%').str.replace('.', '').str.replace(',', '.').astype(float)
data['Date'] = pd.to_datetime(data['Date'], format='%d/%m/%Y')

data = data.sort_values(by='Date')

data = data.rename(columns={'Date': 'ds', 'Dernier': 'y'})

print(data.columns)

model = Prophet()

model.add_regressor('Vol.')

model.fit(data)


15:20:46 - cmdstanpy - INFO - Chain [1] start processing


Index(['ds', 'y', 'Ouv.', ' Plus Haut', 'Plus Bas', 'Vol.', 'Variation %'], dtype='object')


15:20:46 - cmdstanpy - INFO - Chain [1] done processing


In [275]:
nbJours = 300
future_dates = pd.date_range(start=data['ds'].max(), periods=nbJours+1, freq='D')[1:]
future_days = pd.DataFrame({'ds': future_dates})

future_days['Vol.'] = 0 

forecast_next_day = model.predict(future_days)
forecast_previous_days = model.predict(data)

fig = go.Figure()

fig.add_trace(go.Scatter(x=data['ds'], y=data['y'], mode='lines', name='Données réelles', line=dict(color='red')))
# prediction sur données d'entrainement
fig.add_trace(go.Scatter(x=forecast_previous_days['ds'], y=forecast_previous_days['yhat'], mode='lines', name='Predict train', line=dict(color='blue', dash='dash')))

fig.add_trace(go.Scatter(x=forecast_previous_days['ds'], y=forecast_previous_days['yhat_lower'], mode='lines', name='Predict train Lower', line=dict(color='violet', dash='dash')))

fig.add_trace(go.Scatter(x=forecast_previous_days['ds'], y=forecast_previous_days['yhat_upper'], mode='lines', name='Predict train Upper', line=dict(color='green', dash='dash')))
fig.add_trace(go.Scatter(x=forecast_days['ds'], y=forecast_next_day['yhat'], mode='lines', name='Prédictions Model', line=dict(color='blue')))
fig.add_trace(go.Scatter(x=forecast_days['ds'], y=forecast_next_day['yhat_lower'], mode='lines', name='Lower', line=dict(color='violet')))
fig.add_trace(go.Scatter(x=forecast_days['ds'], y=forecast_next_day['yhat_upper'], mode='lines', name='Upper', line=dict(color='green')))
fig.add_trace(go.Scatter(x=forecast_days['ds'], y=forecast_next_day['trend'], mode='lines', name='Trend', line=dict(color='black', dash='dash')))

# Ajouter les axes et le titre
fig.update_layout(xaxis_title='Date', yaxis_title='Prix du Bitcoin (USD)', title=f'Prévisions du prix du Bitcoin avec données réelles et prédictions pour les {nbJours} prochains jours')
fig.show()


In [274]:
forecast_next_day

,ds,trend,yhat_lower,yhat_upper,trend_lower,trend_upper,Vol.,Vol._lower,Vol._upper,additive_terms,...,weekly,weekly_lower,weekly_upper,yearly,yearly_lower,yearly_upper,multiplicative_terms,multiplicative_terms_lower,multiplicative_terms_upper,yhat
0,2024-03-26,51498.131837,53061.438955,64020.210662,51498.131837,51498.131837,-804.454884,-804.454884,-804.454884,7051.554243,...,-54.061150,-54.061150,-54.061150,7910.070277,7910.070277,7910.070277,0.0,0.0,0.0,58549.686081
1,2024-03-27,51581.169836,53267.475367,64241.114619,51581.169836,51581.169836,-804.454884,-804.454884,-804.454884,7105.670649,...,74.389624,74.389624,74.389624,7835.735909,7835.735909,7835.735909,0.0,0.0,0.0,58686.840485
2,2024-03-28,51664.207835,52328.693958,64196.918805,51664.207835,51664.207835,-804.454884,-804.454884,-804.454884,6858.176473,...,-81.370431,-81.370431,-81.370431,7744.001789,7744.001789,7744.001789,0.0,0.0,0.0,58522.384308
3,2024-03-29,51747.245833,52734.203546,64025.092308,51747.245833,51747.245833,-804.454884,-804.454884,-804.454884,6785.293482,...,-46.742192,-46.742192,-46.742192,7636.490558,7636.490558,7636.490558,0.0,0.0,0.0,58532.539315
4,2024-03-30,51830.283832,52816.787154,64616.872208,51830.283832,51830.283832,-804.454884,-804.454884,-804.454884,6763.115547,...,52.459380,52.459380,52.459380,7515.111050,7515.111050,7515.111050,0.0,0.0,0.0,58593.399379
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
85,2024-06-19,58556.361729,48703.093006,61264.213276,55578.464591,61037.006429,-804.454884,-804.454884,-804.454884,-3615.704111,...,74.389624,74.389624,74.389624,-2885.638851,-2885.638851,-2885.638851,0.0,0.0,0.0,54940.657618
86,2024-06-20,58639.399728,48322.732244,60952.097946,55618.572916,61158.664683,-804.454884,-804.454884,-804.454884,-3812.338809,...,-81.370431,-81.370431,-81.370431,-2926.513494,-2926.513494,-2926.513494,0.0,0.0,0.0,54827.060919
87,2024-06-21,58722.437727,48221.951530,61203.012113,55660.059775,61278.378506,-804.454884,-804.454884,-804.454884,-3818.868908,...,-46.742192,-46.742192,-46.742192,-2967.671832,-2967.671832,-2967.671832,0.0,0.0,0.0,54903.568818
88,2024-06-22,58805.475725,48516.423765,61285.697603,55699.827260,61402.734632,-804.454884,-804.454884,-804.454884,-3761.510238,...,52.459380,52.459380,52.459380,-3009.514734,-3009.514734,-3009.514734,0.0,0.0,0.0,55043.965487
